# Radial Basics Functions a Recurrent Neural Network 

Minule jsme si ukazovali a zkoušeli si naprogramovat jednoduché neuronové sítě. Dneska se podíváme na složitější struktury neuronových sítí Radial Basics Functions a rekurentní neuronové sítě.


## Radial Basics Functions (RBF)

Naimplementujeme si nyní RBF síť. Implementace v tensorflow je jednoduchá -- stačí naimplementovat třídu, která bude mít dvě metody. Metoda *build* nainicializuje parametry podle velikosti vstupu a metoda *call* pak implementuje vlastní výpočet.

In [1]:
import tensorflow as tf
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
import numpy as np
import random
import sys

C:\Users\katie\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class RBFNetwork():
    def __init__(self, input_dim, num_centers, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_centers = num_centers
        self.centers = [np.random.uniform(-1, 1, input_dim) for i in range(num_centers)]
        self.beta = 1
        self.weights = None
        
    # spocteni hodnot neuronu na skryte vrstve
    def calculate_activation(self, data):
        hidden_layer_values = np.zeros((data.shape[0], self.num_centers), float)
        for c_idx, c in enumerate(self.centers):
            for x_idx, x in enumerate(data):
                hidden_layer_values[x_idx,c_idx] = self.activation_fcn(c, x)
        return hidden_layer_values
    
    #  hodnota aktivacni fce
    def activation_fcn(self, centers, data):
        return np.exp(-self.beta * np.linalg.norm(centers-data)**2)
    
    def fit(self, data, labels):
        # zvol nahodne hodnoty pocatecnich centroidu
        random_idx = np.random.permutation(data.shape[0])[:self.num_centers]
        self.centers = [data[i,:] for i in random_idx]
         
        # spocitame aktivaci mezi vstupem a skrytou vrstvou
        hidden_layer_values = self.calculate_activation(data)
         
        # porovname skutecne a predikovane vystupy a aktualizujem vahy 
        # pseudoinverzni matice je vlastne vzorecek pro LR pro train vah
        self.weights = np.dot(np.linalg.pinv(hidden_layer_values), labels)
          
    def predict(self, data):
        hidden_layer_values = self.calculate_activation(data)
        labels = np.dot(hidden_layer_values, self.weights)
        return labels


Zkusíme si naši RBF síť pustit na našem oblíbeném datasetu Iris. Načteme si data a labely, do které třídy data patří. Protože budeme dělat klasifikaci, je vhodné si labely převést na one-hot-encoding. Následně data rozdělíme na trénovací a testovací množinu, abychom mohli zvlášť data trénovat a zvlášť data testovat.

In [3]:
# ziskame data
iris = datasets.load_iris()
x, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# delame klasifikaci prevedeme vystup do one hot encoding
onehotencoder = OneHotEncoder(categorical_features = [0]) 
y_train_onehot = onehotencoder.fit_transform(y_train.reshape(-1, 1)).toarray() 
y_test_onehot = onehotencoder.fit_transform(y_test.reshape(-1, 1)).toarray() 

# natrenujeme a ohodnotime sit
rbf = RBFNetwork(4, 10, 3)
rbf.fit(x_train, y_train_onehot)
predicted = rbf.predict(x_test)
y_pred = np.argmax(predicted, axis=1)
accuracy = np.mean(y_pred == y_test)
print('Accuracy: ' + str(accuracy))

Accuracy: 0.9736842105263158


Vidíme, že accuracy je v každém běhu dosti různá, což je způsobeno tím, že centroidy a betu zde nastavujeme náhodně, přestože jsme si říkali, že pozice centroidů se dá trénovat pomocí algoritmu k-means.

### Úkol na cvičení

Zkuste si naimplementovat algoritmus k-means pro inicializaci středů vstupních neuronů a zlepšit tím výstup sítě výše. Hint: metoda `add_variable` má parametr initializer.


## Rekurentní neuronové sítě

Rekurentní neuronová síť je síť, která navíc ke svému vstupu ještě bere jako další vstup svůj výstup z předchozího kroku. Proto výstupy z předchozí vrstvy mohou ovlivňovat vrstvu následující, což se může hodit například u generování časových řad nebo textu. Nejprve se podíváme, jak přesně by vypadala implementace jednoduché RNN. Informace z předchozího kroku se ukládá v RNN buňce do speciální proměnné -- tzv. skrytého stavu, který nám pak bude ovlivňovat další výstupy a bude se s každým novým vstupem v každém kroku aktualizovat.

In [4]:
class RNN:
    def step(self, x):
    # update skrytého stavu
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # vypocet vystupniho vektoru
        y = np.dot(self.W_hy, self.h)
        return y

rnn = RNN()
y = rnn.step(x) 

AttributeError: 'RNN' object has no attribute 'W_hh'

##  Sekvenční klasifikace pomocí LSTM

Nyní, když chápeme, jak taková základní RNN funguje, zkusíme se podívat na složitější druh RNN -- LSTM sítě. Tyto sítě mají navíc uvnitř sebe paměťovou buňku a mechanizmus, který řídí,  jakou informaci si buňka pamatuje a jakou zapomíná. Zkusíme se na ni lépe podívat v následujícím příkladu sekvenční klasifikace. 

Sekvenční klasifikace je prediktivní modelovací problém, kdy máme na vstupu nějakou sekvenci v prostoru nebo čase a cílem je předpovědět kategorii této sekvence. Složitost tohoto problému spočívá v tom, že jednotvlivé sekvence mohou mít různou délku nebo mohou být složeny z rozsáhlého slovníku vstupních hodnot a mohou vužadovat, aby se model naučil nějaké dlouhodobé závislosti nebo kontext mezi vstupními sekvencemi.

Zkusíme se tedy podívat na příklad sekvenční klasifikace pomocí LSTM na IMDB datasetu, což je dataset, který obsahuje slovní popis recenzí 50K filmů a následně klasifikaci, jestli byla recenze pozitivní nebo negativní v poměru 1:1. 

In [31]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# zafixujem random seed pro reprodukovatelnost
numpy.random.seed(7)

Problém je, že slovní popis je nějak potřeba převést na číselnou reprezentaci. Naštěstí funkce ```imdb.load_data``` umí načíst data tak, že rovnou slova nahradí čísly a rozdělí je na train a test množiny v poměru 1:1. Navíc data načteme tak, že necháme jen prvních ```top_words``` nejčastějších slova  zbytek nahradíme 0. Dále je potřeba zkrátit nebo doplnit vstupní sekvence pro modelování tak, aby byly všechny stejně dlouhé, délku nastavíme na ```max_len```.

In [28]:
top_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

max_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)

Nyní máme připravená data a můžeme si definovat a natrénovat model.
 - První vrstva je ```Embedding```, která používá vektory délky 32 pro každé slovo. 
 - Další vrstva je ```LSTM``` vrstva, která obsahuje 100 paměťových jednotek (neuronů). 
 - Na závěr použijeme ```Dense``` výstupní vrstvu s jedním neuronem a aktivační funkcí sigmoid k vytvoření predikcí 0 nebo 1, protože se jedná o klasifikační úlohu.

Problém modelu je, že se velice snadno overfittuje na na daná trénovací data. Proto se se používají ještě vrstvy ```Dropout```, které spočívají v tom, že během trénování se náhodně vynechávají některé vstupy do další vrstvy. Tím se simuluje velký počet sítí s odlišnou strukturou a uzly jsou pak robustnější a omezuje se tím pádem overfitting.

In [32]:
# create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 484s 19ms/step - loss: 0.4778 - acc: 0.7633
Epoch 2/3
25000/25000 [=======

Na závěr zkusíme predikovat výstupy na testovacích datech a podívat se, jak je model dobrý. Můžete si třeba zkusit pustit trénování modelu s droupoutem a bez něj a podívat se, jak se budou lišit výsledné accuracies.

In [33]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.14%


## Generování textu znak po znaku 

Nyní se podíváme na jiný druh problému -- budeme generovat text znak po znaku, neboli natrénujeme jazykový model tak, že když mu pak dáme sekvenci znaků, tak nám model bude schopný předpovědět další znak. Jako trénovací množinu použijeme texty Nietzscheho.

In [ ]:
'''
    Example script to generate text from Nietzsche's writings.
    At least 20 epochs are required before the generated text
    starts sounding coherent.
    It is recommended to run this script on GPU, as recurrent
    networks are quite computationally intensive.
    If you try this script on new data, make sure your corpus
    has at least ~100k characters. ~1M is better.
'''

# nacteme si vstupni data
path = tf.keras.utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# rozdelime text na castecne zavisle sekvence znaku delky maxlen
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# prevedeme text na ciselne vektory
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# vytvorime si model    
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation=tf.nn.softmax))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# pomocna funkce k ziskani indexu z pravdepodobnostniho pole
def sample(a, temperature=1.0):  
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    a = a/np.sum(a)
    return np.argmax(np.random.multinomial(1, a, 1))

# natrenujeme model a po kazde iterace generujeme vystup
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for _ in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Skript si samozřejmě můžete pustit, ale trénování poběží neskutečně dlouho. Proto jsme skript pustili na Google Colab a na výsledky se můžete podívat [zde](https://colab.research.google.com/drive/1B7zys275xmpPqahPwNvuYMPLmgvlV3l5) nebo [zde](/notebooks/07_rbf_rnn/results.txt).